In [1]:
import sys
import pandas as pd
import numpy as np
import quandl
import os
import warnings
from datetime import datetime
from tqdm import tqdm_notebook as tqdm
import pandas_datareader as pdr
import matplotlib.pyplot as plt

quandl.ApiConfig.api_key = os.getenv('QUANDL_APY_KEY')

In [2]:
#columns = ['calendardate', 'datekey', 'revenue', 'currentratio', 'eps', 'dps', 'bvps', 'price']
df = quandl.get_table('SHARADAR/TICKERS', paginate=True) #, dimension='ARY', ticker=ticker), qopts={"columns":columns})
df.head()

,table,permaticker,ticker,name,exchange,isdelisted,category,cusips,siccode,sicsector,...,currency,location,lastupdated,firstadded,firstpricedate,lastpricedate,firstquarter,lastquarter,secfilings,companysite
None,,,,,,,,,,,,,,,,,,,,,
0,SF1,196290,A,Agilent Technologies Inc,NYSE,N,Domestic,00846U101,3826.0,Manufacturing,...,USD,California U.S.A,2018-12-20,2014-09-26,1999-11-18,2019-01-25,1997-06-30,2018-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.agilent.com
1,SF1,124392,AA,Alcoa Corp,NYSE,N,Domestic,013872106,3350.0,Manufacturing,...,USD,New York U.S.A,2018-11-02,2016-11-01,2016-11-01,2019-01-25,2014-12-31,2018-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.alcoa.com
2,SF1,122827,AAAB,Admiralty Bancorp Inc,NASDAQ,Y,Domestic,007231103,6022.0,Finance Insurance And Real Estate,...,USD,Florida U.S.A,2018-10-16,2017-09-09,1998-09-28,2003-01-29,1997-09-30,2002-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,None
3,SF1,120538,AAAGY,Altana Aktiengesellschaft,NYSE,Y,ADR,02143N103,2834.0,Manufacturing,...,EUR,Jordan,2018-02-13,2018-02-13,2002-05-22,2010-08-12,2000-12-31,2005-12-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,None
4,SF1,155760,AAAP,Advanced Accelerator Applications SA,NASDAQ,Y,ADR,00790T100,2834.0,Manufacturing,...,EUR,France,2017-07-17,2016-05-19,2015-11-11,2018-02-09,2013-12-31,2016-12-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,None


In [3]:
df.shape

(34338, 28)

In [4]:
df.columns

Index(['table', 'permaticker', 'ticker', 'name', 'exchange', 'isdelisted',
       'category', 'cusips', 'siccode', 'sicsector', 'sicindustry',
       'famasector', 'famaindustry', 'sector', 'industry', 'scalemarketcap',
       'scalerevenue', 'relatedtickers', 'currency', 'location', 'lastupdated',
       'firstadded', 'firstpricedate', 'lastpricedate', 'firstquarter',
       'lastquarter', 'secfilings', 'companysite'],
      dtype='object')

In [5]:
#Checking survival bias
print(f"{df[df['isdelisted'] == 'N'].shape[0]:,d} companies alive from {df.shape[0]: ,d} total included, all tables")
print(f"{df[(df['table'] == 'SF1') & (df['isdelisted'] == 'N')].shape[0]: ,d} companies alive from {df[df['table'] == 'SF1'].shape[0]: ,d} total included, table SF1")

14,710 companies alive from  34,338 total included, all tables
 5,260 companies alive from  14,121 total included, table SF1


In [8]:
def get_data(ticker, dt=datetime.today(), force_download=False):
    fname = os.path.join(os.getcwd(), 'data_defensive', ticker.lower() + '.csv')
    if force_download or not os.path.isfile(fname):
        columns = ['calendardate', 'datekey', 'revenueusd', 'currentratio', 'eps', 'dps', 'bvps', 'price']
        df = quandl.get_table('SHARADAR/SF1', dimension='ARY', ticker=ticker, qopts={"columns":columns})
        df.sort_values('calendardate', inplace=True)
        df.to_csv(fname)
    else:
        df = pd.read_csv(fname, parse_dates=['calendardate', 'datekey'])

    df = df[df['datekey'] <= dt]
    df = df.groupby('calendardate').tail(1)
    return df

In [9]:
list_tickers = df[df['table'] == 'SF1']['ticker'].tolist()
for ticker in tqdm(list_tickers):
    get_data(ticker, force_download=True)

QuandlError: (Status 522) Something went wrong. Please try again. If you continue to have problems, please contact us at connect@quandl.com.